# Documentation: Java Game Project
Wintersemester 2025/26 – Hochschule Bielefeld – 
Author: Manuel Borghardt, Darnell Borghardt, Joel Jantschik, Leonid Nikkel

---

## 1. Projektidee & Konzept
- Spielfigur sammelt Coins und weicht Hindernissen aus. Wobei die Coins und Hindernisse von oben fallen.
- Ziel: Spieler muss im ersten Level 20 Coins und im zweiten Level 50 Coins Einsammeln. Darf dabei aber nicht einmal vom Hinderniss getroffen werden.
- Steuerung mit Pfeiltasten und 'A' und 'D', es gibt zwei Level wie schon im Ziel erwähnt.

---

## 2. Entwicklungsumgebung
- Java-Version: SE 17
  - Version SE 17, weil dann die Arbeit mit libGDX richtig funktioniert
- Framework/Library: libGDX
- Tools: Visual Studio Code mit folgenden Extensions:
  - Java
  - Extension Pack for Java
  - Gradle for Java
  - Jupyter

---

## 3. LLM
- Copilot

---

## 4. GUI
### 4.1 Spielfeld
- Prompt in Copilot:
  - Projektidee erklärt und gesagt das mit libGDX gearbeitet wird
  - Main.java und Player.java erstellen
- Beschreibung:  
  - Fenstergröße: 1000 × 1000 Pixel.  
  - Hintergrundfarbe: dunkelblau/grau (`glClearColor(0.1f, 0.1f, 0.2f, 1)`).  
  - Spielfigur: grünes Quadrat (60 × 70 Pixel).  
  - Punktestand: Textanzeige oben links.  

- Code der von Copilot für Main.java genutzt wurde:
```java
public class Main extends ApplicationAdapter {
    private OrthographicCamera camera;
    private SpriteBatch batch;
    private ShapeRenderer shapeRenderer;
    private BitmapFont font;

    private Player player;

    @Override
    public void create() {
        // Kamera einrichten
        camera = new OrthographicCamera();
        camera.setToOrtho(false, 1000, 1000);

        // Renderer und Schriftart initialisieren
        batch = new SpriteBatch();
        shapeRenderer = new ShapeRenderer();
        font = new BitmapFont();

        // Spieler initialisieren
        player = new Player(480);
    }
    // Render-Schleife
    @Override
    public void render() {
        handleInput();

        // Hintergrundfarbe setzen
        Gdx.gl.glClearColor(0.1f, 0.1f, 0.2f, 1);
        Gdx.gl.glClear(GL20.GL_COLOR_BUFFER_BIT);

        camera.update();

        // Spieler zeichnen
        shapeRenderer.setProjectionMatrix(camera.combined);
        shapeRenderer.begin(ShapeRenderer.ShapeType.Filled);
        player.render(shapeRenderer);
        shapeRenderer.end();

        // Punktestand anzeigen
        batch.setProjectionMatrix(camera.combined);
        batch.begin();
        font.draw(batch, "Punkte: " + player.getPoints(), 10, 980);
        batch.end();
    }
    // Eingaben verarbeiten
    private void handleInput() {
        if (Gdx.input.isKeyPressed(Input.Keys.LEFT) || Gdx.input.isKeyPressed(Input.Keys.A)) player.moveLeft();
        if (Gdx.input.isKeyPressed(Input.Keys.RIGHT) || Gdx.input.isKeyPressed(Input.Keys.D)) player.moveRight();
    }
    // Ressourcen freigeben
    @Override
    public void dispose() {
        batch.dispose();
        shapeRenderer.dispose();
        font.dispose();
    }
}

### 4.2 Spielfigur(Player)
- Objektorientierte Umsetzung: eigene Klasse Player.
- Attribute: x, y (Position), points (Punktestand).
- Methoden: moveLeft(), moveRight(), moveUp(), moveDown(), render().
- Darstellung: grünes Quadrat, das sich über Tastatureingaben bewegt.
- Code der von Copilot für Player.java genutzt wurde:
```java
public class Player {
    // Spielfigur-Position und Punktestand
    private float x, y;
    private int points;

    // Konstruktor
    public Player(float startX, float startY) {
        this.x = startX;
        this.points = 0;
    }

    // getter für Punktestand
    public int getPoints() { return points; }

    // Bewegung der Spielfigur nach links
    public void moveLeft() { x -= 3; points++; }
    // Bewegung der Spielfigur nach rechts
    public void moveRight() { x += 3; points++; }

    // Spielfigur zeichnen
    public void render(ShapeRenderer renderer) {
        renderer.setColor(0, 1, 0, 1);
        renderer.rect(x, y, 60, 70);
    }
}

### Erstes Fenster mit Spielfigur
<img src="./Erstes%20Fenster.png" alt="Erstes Fenster" width="400"/>

## 5. Bug-fix
- Spielfigur konnte links und rechts aus dem Window verschwinden
- Lösung (selber gemacht):
```java
// Bewegung der Spielfigur nach links
public void moveLeft() {
    x -= 3;
    if(x == 0) {
        x += 3;
    }
}

// Bewegung der Spielfigur nach rechts
public void moveRight() {
    x += 3;
    if(x == 939) {
        x -= 3;
    }
}

## 6. Coins und Enemies

### Coins
- Verhalten: Coins spawnen periodisch oben an einer zufälligen X-Position und fallen nach unten.
- Standard-Parameter (in der Implementierung):
  - Spawn-Intervall: `0.8s`
  - Größe: `20 x 20` Pixel
  - Fallgeschwindigkeit: `200 px/s`
- Lebenszyklus:
  - `spawnCoin()` erzeugt neue Coins
  - `update(delta)` bewegt jeden Coin (`y -= speed * delta`)
  - Coins werden entfernt, sobald sie vollständig unterhalb des Bildschirms sind (`y + height < 0`)
  - Bei Kollision mit dem Player (`collectCollisions(player)`) wird der Coin entfernt und der Punktestand erhöht (`player.addPoints(1)`).

Wichtige Methoden / Aufrufe im Spiel:
```java
// in Main.create()
coins = new Coins(1000, 1000);

// in Main.render()
float delta = Gdx.graphics.getDeltaTime();
coins.update(delta);
coins.collectCollisions(player);
// zeichnen
coins.render(shapeRenderer);
```

### Enemy
- Verhalten: Enemys verhalten sich ähnlich wie Coins, haben aber andere Parameter und bewirken einen Punktabzug beim Kontakt.
- Standard-Parameter (in der Implementierung):
  - Spawn-Intervall: `2.6s`
  - Größe: `30 x 25` Pixel
  - Fallgeschwindigkeit: `180 px/s`
- Lebenszyklus:
  - `spawnEnemy()` erzeugt neue Enemys
  - `update(delta)` bewegt den Enemy nach unten
  - Enemys werden entfernt, sobald sie unterhalb des Bildschirms sind
  - Bei Kollision mit dem Player (`badCollisions(player)`) wird der Enemy entfernt und der Player verliert einen Punkt (`player.addPoints(-1)`).

Wichtige Methoden / Aufrufe im Spiel:
```java
// in Main.create()
enemy = new Enemy(1000, 1000);

// in Main.render()
enemy.update(delta);
enemy.badCollisions(player);
// zeichnen
enemy.render(shapeRenderer);
```

### Integration in `Main` (Coins + Enemy)
- Initialisierung erfolgt in `create()` mit der Welt-/Kamera-Größe, damit Spawn-Positionen korrekt berechnet werden.
- Im Render-Loop werden zuerst `update(delta)` für Coins und Enemys aufgerufen, dann die Kollisionsprüfungen (`collectCollisions`, `badCollisions`), und zuletzt werden alle Objekte mit demselben `ShapeRenderer` gezeichnet.

Beispiel (Ausschnitt aus `Main.render()`):
```java
float delta = Gdx.graphics.getDeltaTime();

// Coins
coins.update(delta);
coins.collectCollisions(player);

// Enemys
enemy.update(delta);
enemy.badCollisions(player);

// Rendern (ShapeRenderer-Durchlauf)
shapeRenderer.begin(ShapeRenderer.ShapeType.Filled);
coins.render(shapeRenderer);
enemy.render(shapeRenderer);
player.render(shapeRenderer);
shapeRenderer.end();
```
